In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import shutil
source_dir = '/content/drive/MyDrive/SecurityNetwork/Dataset/pcap_dataset'
destination_dir = '/content/pcap_dataset'

shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

'/content/pcap_dataset'

In [ ]:
# @title Brimcap extract
!curl -OL https://github.com/brimdata/zed/releases/download/v1.11.1/zed-v1.11.1.linux-amd64.tar.gz
!curl -OL https://github.com/brimdata/brimcap/releases/download/v1.5.4/brimcap-v1.5.4.linux-amd64.zip
!tar xzvf zed-v1.11.1.linux-amd64.tar.gz
!chmod +x *
!unzip brimcap-v1.5.4.linux-amd64.zip
%cd brimcap

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 23.3M  100 23.3M    0     0  24.7M      0 --:--:-- --:--:-- --:--:-- 51.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 81.4M  100 81.4M    0     0  94.0M      0 --:--:-- --:--:-- --:--:--  211M
LICENSE.txt
acknowledgments.txt
zed
zq
Archive:  brimcap-v1.5.4.linux-amd64.zip
   creating: brimcap/
  inflating: brimcap/brimcap         
   creating: brimcap/zeek/
   creating: brimcap/zeek/bin/
  inflating: brimcap/zeek/bin/zeek   
   creating: brimcap/zeek/share/
   creating: brimcap/zeek/share/zeek/
   creating: brimcap/zeek/share/zeek/site/
   creating: brimc

In [ ]:
# @title Phân tích file pcap
sh="""
ListFileAndExtract(){
#duyệt file và đệ quy
  for file in $1*.pcap
  do
    if [ ! -f $file ]
    then
        ListFileAndExtract "$file/*" "$file"
    else
#Phân tích file pcap thành nhật ký zeek và chuyển thành các log
        echo brimcap:
        ./brimcap analyze $file > $1extract.zng
#chia các feature
        echo zq:
        .././zq -z '_path=="ssl"' $1extract.zng > ${file::-5}-ssl.zng
        .././zq -f 'csv' ${file::-5}-ssl.zng > ${file::-5}-ssl.csv
        .././zq -z '_path=="conn"' $1extract.zng > ${file::-5}-conn.zng
        .././zq -f 'csv' ${file::-5}-conn.zng > ${file::-5}-conn.csv
        .././zq -z '_path=="x509"' $1extract.zng > ${file::-5}-x509.zng
        .././zq -f 'csv' ${file::-5}-x509.zng > ${file::-5}-x509.csv
#xóa file không cần thiết
        rm ${file::-5}-ssl.zng ${file::-5}-conn.zng ${file::-5}-x509.zng

    fi
  done
}
ListFileAndExtract "/content/pcap_dataset/"

"""
with open('ExtractPcap.sh','w') as file:
  file.write(sh)
!bash ExtractPcap.sh

brimcap:
  0.0% 65.54KB/294.70MB values=0 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.0% 65.54KB/294.70MB values=83 
  0.5% 1.61M

In [ ]:
# @title Merging data

import pandas as pd
from google.colab import files
import requests
import os
import re
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
# Check if the certificate was registered at letsencrypt.org
def check_certificate_registration(domain):
    # Make a GET request to the LetsEncrypt certificate transparency API
    response = requests.get('https://crt.sh/?i={}'.format(domain))

    # Check if the response status code is 200
    if response.status_code == 200:
        return True
    else:
        return False
list_family = ['normal', 'attack', 'label']
def get_family(string):
  for family in list_family:
    match = re.search(r'\b\w*normal\w*\b', string)
    if match:
       return match.group()
    else:
        return None

def Clean_data(df, target_variable):
    print("Start cleanning data")

    # Remove duplicated samples (rows)
    df = df.drop_duplicates()
    # Impute missing data or infinite values with mean value of each feature
    if target_variable is None:
        X = df
    else:
        X = df.drop(target_variable, axis=1)

    # Label Encoding
    for col in X.columns:
      if X[col].dtypes == 'object':
        le = LabelEncoder()
        X[col]=X[col].astype('str')
        X[col] = le.fit_transform(X[col])

    # Replace the NaN value to the average value of the column
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    imputer = SimpleImputer(strategy='mean')
    X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

    # Concatenate the target variable (if any) and the reduced feature DataFrame
    if target_variable in df.columns:
        y = df[target_variable]
        y.reset_index(drop=True, inplace=True)
        X.reset_index(drop=True, inplace=True)
        X = pd.concat([X, y], axis=1)
    return X

df = pd.DataFrame()
if os.path.exists("Dataset"):
    os.mkdir("/content/Dataset/")
for x in os.listdir("/content/pcap_dataset"):
    if x.endswith(".pcap"):
        # Prints only text file present in My Folder
# Read the three CSV files
      df_conn = pd.read_csv("/content/pcap_dataset/"+x[:-5]+'-conn.csv')
      df_ssl = pd.read_csv("/content/pcap_dataset/"+x[:-5]+'-ssl.csv')
      df_x509 = pd.read_csv("/content/pcap_dataset/"+x[:-5]+'-x509.csv')

# Convert 'server_name' to 'server_name_len' in df_ssl
      df_ssl['server_name_len'] = df_ssl['server_name'].astype(str).apply(len)

# Convert 'san.dns' to 'san_dns_num' in df_x509 for the length of DNS
      df_x509['san_dns_num'] = df_x509['san.dns'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

# Convert 'san.ip' to 'san_ip_num' in df_x509 for the length of IP address
      df_x509['san_ip_num'] = df_x509['san.ip'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

# Extract 'domain_name' in df_x509
      df_x509['domain_name'] = df_x509['certificate.subject'].str.extract(r'CN=.*?\.([^,]+)')
      df_x509['lets_encrypt'] = (
      (check_certificate_registration(df_x509['domain_name']))
      )

# Extract 'first_value' from 'cert_chain_fuids' in df_ssl
      df_ssl['first_value'] = df_ssl['cert_chain_fuids'].apply(lambda x: x.split('"')[1] if isinstance(x, str) else None)

# Extract specific columns from each DataFrame
      columns_to_extract_df_conn = ['uid', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'history', 'missed_bytes', 'orig_pkts', 'resp_ip_bytes', 'resp_pkts', 'orig_ip_bytes']
      columns_to_extract_df_ssl = ['uid', 'version', 'cipher', 'resumed', 'established', 'server_name_len', 'next_protocol', 'validation_status', 'cert_chain_fuids', 'first_value']
      columns_to_extract_df_x509 = ['id', 'certificate.version', 'san_dns_num', 'san_ip_num', 'lets_encrypt', 'certificate.key_alg', 'certificate.sig_alg', 'certificate.key_type', 'certificate.key_length', 'basic_constraints.ca', 'certificate.curve']

      df_conn_selected = df_conn[columns_to_extract_df_conn]
      df_ssl_selected = df_ssl[columns_to_extract_df_ssl]
      df_x509_selected = df_x509[columns_to_extract_df_x509]

# Merge df_conn and df_ssl by 'uid'
      merged_conn_ssl = pd.merge(df_conn_selected, df_ssl_selected, on='uid', how='inner')
      merged_result = pd.merge(merged_conn_ssl, df_x509_selected, left_on='first_value', right_on='id', how='inner')

# Drop unnecessary columns
      merged_result = merged_result.drop(columns = ['first_value', 'uid', 'id', 'cert_chain_fuids'])
# Add family
      merged_result['family'] = get_family(x)
# Download file after processing
      # merged_result()
      print(f"==========================================={get_family(x)}======================")
      df = pd.concat([df, merged_result])
      print(merged_result)
      # files.download(x[:-5]+'.csv')

df = Clean_data(df, target_variable='family')

===========================================normal======================
     proto service      duration  orig_bytes  resp_bytes conn_state  \
0      tcp     ssl     5.321756s       373.0      3710.0         SF   
1      tcp     ssl     1.438172s       857.0     47461.0         SF   
2      tcp     ssl     119.855ms       500.0      4341.0       RSTO   
3      tcp     ssl     211.529ms       496.0      4407.0         SF   
4      tcp     ssl     5.157124s       373.0      3710.0         SF   
...    ...     ...           ...         ...         ...        ...   
2245   tcp     ssl    27.839263s      1061.0      5157.0         SF   
2246   tcp     ssl   2m1.537483s      1884.0      5585.0         SF   
2247   tcp     ssl    3m15.6694s       980.0      4453.0         SF   
2248   tcp     ssl  1m41.851565s      1438.0     49785.0         SF   
2249   tcp     ssl  1m47.816504s      7492.0     23745.0         SF   

         history  missed_bytes  orig_pkts  resp_ip_bytes  ...  san_dns_num 

In [ ]:
# df.to_csv('/content/drive/MyDrive/SecurityNetwork/final_dataset.csv',index=False)

proto service      duration  orig_bytes  resp_bytes conn_state  \
0      tcp     ssl     5.321756s       373.0      3710.0         SF   
1      tcp     ssl     1.438172s       857.0     47461.0         SF   
2      tcp     ssl     119.855ms       500.0      4341.0       RSTO   
3      tcp     ssl     211.529ms       496.0      4407.0         SF   
4      tcp     ssl     5.157124s       373.0      3710.0         SF   
...    ...     ...           ...         ...         ...        ...   
1581   tcp     ssl  2m29.701668s      1805.0      5241.0         S1   
1582   tcp     ssl  2m56.985395s      1613.0      4698.0         SF   
1583   tcp     ssl   1m1.691821s      1047.0      5790.0         S1   
1584   tcp     ssl      785.74ms       739.0    225178.0         S1   
1585   tcp     ssl  1m50.764633s      1605.0     14418.0         S1   

         history  missed_bytes  orig_pkts  resp_ip_bytes  ...  san_dns_num  \
0       ShADadFf             0          9           4114  ...           39   
1       ShADadFf             0         24          50305  ...          991   
2       ShADadFR             0          9           4785  ...          991   
3     ShADadFfRr             0         11           5011  ...          991   
4       ShADadFf             0          9           4114  ...           39   
...          ...           ...        ...            ...  ...          ...   
1581      ^hADad             0         21           6349  ...          229   
1582   ShADadfFr             0         25           5730  ...          229   
1583      ShADad             0         24           6890  ...          229   
1584     ShADadg          1448         85         232058  ...           37   
1585      ShADad             0         26          15726  ...           37   

      san_ip_num lets_encrypt certificate.key_alg      certificate.sig_alg  \
0              0         True       rsaEncryption  sha256WithRSAEncryption   
1              0         True      id-ecPublicKey  sha256WithRSAEncryption   
2              0         True      id-ecPublicKey  sha256WithRSAEncryption   
3              0         True      id-ecPublicKey  sha256WithRSAEncryption   
4              0         True       rsaEncryption  sha256WithRSAEncryption   
...          ...          ...                 ...                      ...   
1581           0        False       rsaEncryption  sha256WithRSAEncryption   
1582           0        False       rsaEncryption  sha256WithRSAEncryption   
1583           0        False       rsaEncryption  sha256WithRSAEncryption   
1584           0        False       rsaEncryption  sha256WithRSAEncryption   
1585           0        False       rsaEncryption  sha256WithRSAEncryption   

      certificate.key_type  certificate.key_length basic_constraints.ca  \
0                      rsa                    2048                False   
1                    ecdsa                     256                False   
2                    ecdsa                     256                False   
3                    ecdsa                     256                False   
4                      rsa                    2048                False   
...                    ...                     ...                  ...   
1581                   rsa                    2048                False   
1582                   rsa                    2048                False   
1583                   rsa                    2048                False   
1584                   rsa                    2048                False   
1585                   rsa                    2048                False   

     certificate.curve  family  
0                  NaN  normal  
1           prime256v1  normal  
2           prime256v1  normal  
3           prime256v1  normal  
4                  NaN  normal  
...                ...     ...  
1581               NaN  normal  
1582               NaN  normal  
1583               NaN  normal  
1584               NaN  normal  
1585          